## Problem 1

The data for this problem comes from a subset of The National Centers for Environmental Information (NCEI) [Global Historical Climatology Network daily (GHCNd)](https://www.ncei.noaa.gov/products/land-based-station/global-historical-climatology-network-daily) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe - it's a wonderfully large dataset to play with! In particular, you will be asked to use data from the Ann Arbor Michigan location, and this is stored in the file: [assets/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv](assets/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv).

Each row in this datafile corresponds to a single observation from a weather station, and has the following variables:

- __id__: station identification code
- __date__: date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
- __element__: indicator of element type
    - _TMAX_: Maximum temperature (tenths of degrees F)
    - _TMIN_: Minimum temperature (tenths of degrees F)
- __value__: data value for element (tenths of degrees F)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write a python notebook which plots line graphs of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015. (Based on the graph, do you think extreme weather is getting more frequent in 2015?)
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

Your final output should looks like next one:

![](chris_sketch.png)

# Workshop 3

After you learn about _regular expressions_ and _data manipulation_, it is momment to validate how many do you learn about __data visualization__.  

### Step 1.1

Load the dataset and transform the data into _Celsius_ (refer to documentation). Then extract all of the rows which have minimum or maximum temperatures, each one in a separate dataframe.

__Hint: when I did this step I had two DataFrame objects, each with ~80,000 entries in it__

In [5]:
import pandas as pd

def farenheit_to_celsius(farenheit):
    # YOUR CODE HERE
    return ((farenheit-32)*5)/9

# load the data
data = pd.read_csv("assets/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv",)
data["Data_Value"] = data["Data_Value"].apply(lambda x: farenheit_to_celsius(x))
# YOUR CODE HERE
data['Year'] = pd.to_datetime(data['Date']).dt.year
data['Month-Day'] = pd.to_datetime(data['Date']).dt.strftime('%m-%d')
temperature_min_df = data[(data["Element"]=="TMIN")&(data["Month-Day"]!="02-29")]
temperature_max_df = data[(data["Element"]=="TMAX")&(data["Month-Day"]!="02-29")]

print(temperature_max_df.shape,temperature_min_df.shape)

(83020, 6) (81982, 6)


In [ ]:
# space for proffessor's tests


### Step 1.2

In order to visualize the data we would _plot_ the __min__ and _max_ data for each day of the year between the years $2005$ and $2014$ across all weather stations. But we also need to find out _when_ the __min__ or __max__ temperature in $2015$ falls below the min or rises above the max for the _previous decade_.

If you did _Step 1.1_ you have _two Series_ objects with _min_ and _max_ times for the years $2005$ through $2015$. You can use Pandas `groupby` to create _max_ and _min_ temperature _Series_ objects across all weather stations for each day of these years, and you can deal with the records for _February 29_ (the leap year) by dropping them.

__Hint: When I finished this step, I had two DataFrame objects, each with exactly 4015 observations in them__

In [6]:
# create a DataFrame of maximum temperature by date
# YOUR CODE HERE    
temperature_max_per_day_df = temperature_max_df.groupby('Date')['Data_Value'].max().reset_index()
# create a DataFrame of minimum temperatures by date
# YOUR CODE HERE 
temperature_min_per_day_df = temperature_min_df.groupby('Date')['Data_Value'].min().reset_index()
print( temperature_max_per_day_df.shape,temperature_min_per_day_df.shape)

(4015, 2) (4015, 2)


In [ ]:
# space for proffessor's tests

### Step 1.3
Now that you have grouped the _daily_ __max__ and __min__ temperatures for each day of the years $2005$ through $2015$, you can separate out the data for $2015$. Then you can use the Pandas `groupby` function to find the __max__ and __min__ of the temperature data for each __day of the year__ for the $2005-2014$ data.

__Hint: At the end of this step I had two DataFrames, one of maximum and the other of minimum values, which each had 365 observations in them. I also had another pair of similar DataFrames but only for the year 2015.__

In [10]:
# calculate the minimum and maximum values for the day of the year for 2005 through 2014
# YOUR CODE HERE
data_max_2005_2014 = temperature_max_df[(temperature_max_df['Year'] >= 2005) & (temperature_max_df['Year'] <= 2014)]
data_min_2005_2014 = temperature_min_df[(temperature_min_df['Year'] >= 2005) & (temperature_min_df['Year'] <= 2014)]

temperature_max_day_year_df = data_max_2005_2014.groupby('Month-Day')['Data_Value'].max().reset_index()
temperature_min_day_year_df = data_min_2005_2014.groupby('Month-Day')['Data_Value'].min().reset_index()

# calculate the minimum and maximum values for the years 2015
data_max_2015 = temperature_max_df[temperature_max_df['Year'] == 2015]
data_min_2015 = temperature_min_df[temperature_min_df['Year'] == 2015]
# YOUR CODE HERE
temperature_max_per_day_2015_df = data_max_2015.groupby('Month-Day')['Data_Value'].max().reset_index()
temperature_min_per_day_2015_df = data_min_2015.groupby('Month-Day')['Data_Value'].min().reset_index()

print(temperature_max_day_year_df.shape,temperature_min_day_year_df.shape)
print(temperature_max_per_day_2015_df.shape,temperature_min_per_day_2015_df.shape)

(365, 2) (365, 2)
(365, 2) (365, 2)


In [ ]:
# space for proffessor's tests

### Step 1.4

Now it's time to plot! You need to explore _matplotlib_ in order to plot line graphs of the _min_ and _max_ temperatures for the years $2005$ through $2014$ and to scatter plot __only__ the daily $2015$ temperatures that exceeded those values.

In [11]:
import matplotlib.pyplot as plt
from calendar import month_abbr


fig = plt.figure(figsize=(15, 10))
# YOUR CODE HERE
days = temperature_max_day_year_df['Month-Day']
max_temps_2005_2014 = temperature_max_day_year_df['Data_Value']
min_temps_2005_2014 = temperature_min_day_year_df['Data_Value']

max_temps_2015 = temperature_max_per_day_2015_df['Data_Value']
min_temps_2015 = temperature_min_per_day_2015_df['Data_Value']

# Anomalies
max_exceed_2015 = temperature_max_per_day_2015_df[max_temps_2015 > max_temps_2005_2014]
min_exceed_2015 = temperature_min_per_day_2015_df[min_temps_2015 < min_temps_2005_2014]

fig = plt.figure(figsize=(15, 10))

plt.plot(days, max_temps_2005_2014, label='2005-2014 Max Temperatures', color='red', linewidth=1)
plt.plot(days, min_temps_2005_2014, label='2005-2014 Min Temperatures', color='blue', linewidth=1)

plt.scatter(max_exceed_2015['Month-Day'], max_exceed_2015['Data_Value'], label='2015 Max Temperature Exceeded', color='darkred')
plt.scatter(min_exceed_2015['Month-Day'], min_exceed_2015['Data_Value'], label='2015 Min Temperature Exceeded', color='darkblue')

plt.fill_between(days, min_temps_2005_2014, max_temps_2005_2014, color='grey', alpha=0.4)

tick_positions = [0,31, 59, 90, 120, 151, 181, 212, 243, 273, 304, 334, 364]
custom_month_abbr = ['Year Begins'] + list(month_abbr[1:])
plt.xticks(tick_positions, custom_month_abbr, rotation=45)
plt.xlabel('Day of the Year')
plt.ylabel('Temperature (Â°C)')
plt.title('Daily Maximum and Minimum Temperatures (2005-2014) with 2015 Extremes')
plt.legend()
plt.grid(True)

plt.show()

<Figure size 1500x1000 with 0 Axes>